In [0]:
# Tuning The Recurrent Neural Network To Improve Performance Of An Algorithm

In [0]:
# Importing the Google Stock Price Train Dataset
  import pandas as pd
  dataset_train=pd.read_csv('/content/sample_data/Google_Stock_Price_Train.csv')

  # Splitting the dataset_train into input attribute only
  train_set=dataset_train.iloc[:,1:2].values
  
  # Rescale the train_set google stock price open amount (input attribute) using minmaxscaler class
  from sklearn.preprocessing import MinMaxScaler
  sc=MinMaxScaler()
  train_set_scaled=sc.fit_transform(train_set)


In [0]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout

x_train=[]
y_train=[]
  
# split the Google Stock Price into 60 Timesets and evalute t+1 on train set
for i in range(60,len(train_set)):
   x_train.append(train_set_scaled[i-60:i,])
   y_train.append(train_set_scaled[i,])
# converting numpy vector to numpy array
import numpy as np
x_train,y_train=np.array(x_train),np.array(y_train)
# reshape x_train from 2D to 3D Array
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1)) 

def build_rnn(optimizer):
   

  # Building RNN Algorithm

  # Intialising The RNN Network

  rnn=Sequential()

  # Adding First LSTM Layer and Dropout Regularisation
  rnn.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
  rnn.add(Dropout(p=0.2))

  # Adding Second LSTM Layer and Dropout Regularisation
  rnn.add(LSTM(units=50,return_sequences=True))
  rnn.add(Dropout(p=0.2))

  # Adding Third LSTM Layer and Dropout Regularisation
  rnn.add(LSTM(units=50,return_sequences=True))
  rnn.add(Dropout(p=0.2))

  # Adding Fourth LSTM Layer and Dropout Regularisation
  rnn.add(LSTM(units=50,return_sequences=True))
  rnn.add(Dropout(p=0.2))

  # Adding Fifth LSTM Layer and Dropout Regularisation
  rnn.add(LSTM(units=50))
  rnn.add(Dropout(p=0.2))

  # Adding Output Layer
  from keras.layers import Dense
  rnn.add(Dense(units=1))

  # Compile the RNN Algorithm
  rnn.compile(optimizer=optimizer,loss='mean_squared_error')
  return rnn

In [14]:
# Evaluating and Tuning the RNN Algorithm
rnn=KerasRegressor(build_fn=build_rnn)
params={'optimizer':['adam','rmsprop'],
        'nb_epoch':[25,50,75,100],
        'batch_size':[8,16,32,64]}
# Tuning using GridSearchCV class
from sklearn.model_selection import GridSearchCV
results=GridSearchCV(estimator=rnn,param_grid=params,cv=10,n_jobs=-1,scoring='neg_mean_squared_error')
grid_search=results.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`

Epoch 1/1
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0321


In [15]:
best_parameter=grid_search.best_params_
best_accuracy=grid_search.best_score_
print(best_parameter)
print(best_accuracy)

{'batch_size': 16, 'nb_epoch': 75, 'optimizer': 'adam'}
-0.002678882889088137


In [0]:
# Above Ouput is the Best Combination for batch_size nb_epoch and optimizer